# I - Données, pré-traitement  

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GlobalMaxPooling1D, Bidirectional, Conv1D, concatenate
from keras.models import Model

from tools import *
from embeddings import *
from models import *

Using TensorFlow backend.


In [2]:
# load raw string data
data_train, y_train_all, data_test, id_test = load_data()

## Nettoyage des données (optionnel)

In [3]:
params = {'lower': True, 
          'lemma': False, 
          'stop_words': False}

comment = data_train[2]
print(comment)
print('-------')
print(CommentCleaner(**params).transform(comment))

Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.
-------
hey man i m really not trying to edit war it s just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info 


In [4]:
clean_data_train = transform_dataset(data_train, transformer=CommentCleaner, kwargs=params)
clean_data_test = transform_dataset(data_test, transformer=CommentCleaner, kwargs=params)

Transformation: 100%       
Transformation: 100%       


## Conversion numérique des données textuelles

In [5]:
# Convert strings to int indexes, 
# considering only the VOCAB_SIZE most common words, 
# and pad the sentences to SENTENCE_LENGTH words
VOCAB_SIZE = 30000
## TODO: set parameters in a better way


tfidf_vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 1),
                                   min_df=10, max_features=VOCAB_SIZE, use_idf=1, smooth_idf=1,
                                   sublinear_tf=1)


# X_train_all, X_test = encode(data_train, data_test, vectorizer=tokens_vectorizer)
X_train_all, X_test = encode(clean_data_train, clean_data_test, vectorizer=tfidf_vectorizer)

ENCODING: Fitting vectorizer to data
ENCODING: transforming data to numerical


## Extraction des features auxiliaires

In [6]:
print("Computing comments length")
comments_lengths_train = np.array(transform_dataset(data_train, transformer=CommentLength, n_prints=5))
comments_lengths_test = np.array(transform_dataset(data_test, transformer=CommentLength, n_prints=5))

print("Computing number of punctuation marks in comments")
params = {'divide_by_len': True, 'chars_set': {'!'}}
comments_punctuation_train = np.array(transform_dataset(data_train, transformer=CharCounter, kwargs=params))
comments_punctuation_test = np.array(transform_dataset(data_test, transformer=CharCounter, kwargs=params))

print("Computing number of upper cased words in comments")
params = {'divide_by_len': True}
comments_upperwords_train = np.array(transform_dataset(data_train, transformer=UppercaseWordsCounter, kwargs=params))
comments_upperwords_test = np.array(transform_dataset(data_test, transformer=UppercaseWordsCounter, kwargs=params))

# concatenation of auxiliary features
X_aux_train_all = np.vstack((comments_lengths_train, comments_punctuation_train, comments_upperwords_train)).T
X_aux_test = np.vstack((comments_lengths_test, comments_punctuation_test, comments_upperwords_test)).T

Computing comments length
Transformation: 100%       
Transformation: 100%       
Computing number of punctuation marks in comments
Transformation: 100%       
Transformation: 100%       
Computing number of upper cased words in comments
Transformation: 100%       
Transformation: 100%       


In [7]:
SPLIT_VALID_RATIO = 0.10
SPLIT_RANDOM_SEED = 0  # TODO : check split because of imbalanced classes

# numerical comments
X_train, X_valid, y_train, y_valid = train_test_split(X_train_all, y_train_all, 
                                                      test_size=SPLIT_VALID_RATIO,
                                                      random_state=SPLIT_RANDOM_SEED)

# auxiliary input
X_aux_train, X_aux_valid, _, _ = train_test_split(X_aux_train_all, y_train_all, 
                                                  test_size=SPLIT_VALID_RATIO,
                                                  random_state=SPLIT_RANDOM_SEED)

In [18]:
params = {'C':6*[4],
          'dual': 6*[False],
          'solver': 6*['saga']}
USE_AUX_FEATURES = True

model = OneVAllClassifier(n_classes=6, clf=NbSvmClassifier, params=params)

In [59]:
from scipy.sparse import hstack
test = hstack((X_train, X_aux_train)).astype(int)
# print(np.vstack((X_train, X_aux_train)))
print(type(test))
print(test[:10, :])

<class 'scipy.sparse.coo.coo_matrix'>


TypeError: 'coo_matrix' object is not subscriptable

In [55]:
model.fit(hstack((X_train, X_aux_train)).astype(int) if USE_AUX_FEATURES else X_train, y_train)

TypeError: only integer scalar arrays can be converted to a scalar index

In [15]:
y_train_pred = model.predict_proba([X_train, X_aux_train] if USE_AUX_FEATURES else X_train, y_train)
y_valid_pred = model.predict_proba([X_valid, X_aux_valid] if USE_AUX_FEATURES else X_valid, y_valid)

In [16]:
train_score = evaluate(y_train, y_train_pred)
print("ROC-AUC score on train set : {:.4f}".format(train_score))

valid_score = evaluate(y_valid, y_valid_pred)
print("ROC-AUC score on validation set : {:.4f}".format(valid_score))

ROC-AUC score on train set : 0.9957
ROC-AUC score on validation set : 0.9816


In [17]:
# predict
y_test_pred = model.predict_proba([X_test, X_aux_test] if USE_AUX_FEATURES else X_test)
# write submission file
submission(y_test_pred, id_test, name='NBSVM_aux')    

In [60]:
np.sum(y_test_pred, axis=0)

array([32143.49122533,  2561.32050589, 18416.87495498,   648.40185477,
       14154.11752398,  2475.5881024 ])

In [61]:
np.sum(y_train_pred, axis=0)

array([13790.91974991,  1470.59968963,  7630.25014452,   474.36105437,
        7090.51070723,  1303.86790551])

In [62]:
np.sum(y_train, axis=0)

array([13763,  1434,  7591,   435,  7064,  1258])